In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite:///./app/connection/test.db", echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()

In [4]:
# 이상치 데이터 처리방법
from app.services.kor_value_service import KorValueService


kor_value_service = KorValueService(session)
value_list = kor_value_service.get_values(isDf=True)

In [5]:
value_pbr = value_list[value_list["metrics"] == "PBR"]

In [ ]:
print(value_pbr["amt"].max(), "\n", value_pbr["amt"].min())

In [ ]:
import matplotlib.pyplot as plt

value_pbr["amt"].plot.hist(bins=100, figsize=(10, 6))

In [ ]:
# quantile 분위수에 해당하는 값을 찾아줌( 0.01에 해당하는 값)
q_low = value_pbr["amt"].quantile(0.01)
q_high = value_pbr["amt"].quantile(0.99)

In [ ]:
# trim: 극단치 제거
# 평균이나 분산 같이 통계값을 구하는 과정에서는 이상치 데이터를 제거하는 것이 바람직할 수 있음.
# 팩터를 이용해 포트폴리오를 구하는 과정에서는 트림방법은 조심스럽게 사용되어야함. 데이터의 손실이 발생하기도 하며, 제거된 종목 중 정말로 좋은 종목이 있을 수도 있기 때문

#
value_trim = value_pbr.loc[
    (value_pbr["amt"] > q_low) & (value_pbr["amt"] < q_high), ["amt"]
]

value_trim.plot.hist(figsize=(10, 6), bins=100, legend=False)

In [ ]:
# 윈저라이징: 이상치 데이터를 다른 데이터로 대체
value_winsor = value_pbr[["amt"]].copy()
value_winsor.loc[value_winsor["amt"] < q_low, "amt"] = q_low
value_winsor.loc[value_winsor["amt"] > q_high, "amt"] = q_high

fig, ax = plt.subplots(figsize=(10, 6))
n, bins, patches = plt.hist(value_winsor, bins=100)